In [1]:
from binascii import hexlify
import sys

import numpy as np
import estraces
from rainbow.generics import rainbow_arm
from rainbow import TraceConfig, HammingWeight, Print
from tqdm.notebook import tqdm

sys.path.append('../../../')

from kresca.plot import CurvePlot

## Create Rainbow Object

In [2]:
# e = rainbow_arm(print_config=Print.Code | Print.Functions, trace_config=TraceConfig(register=HammingWeight()))
e = rainbow_arm(trace_config=TraceConfig(register=HammingWeight()))
e.load("des.elf", typ=".elf")
e.setup()

In [3]:
def des_encrypt(key, plaintext):
    e.reset()
    
    key_addr = 0xDEAD0000
    buf_in = 0xDEAD2000
    buf_out = 0xDEAD4000
    
    e[key_addr] = key.tobytes()
    e[buf_in] = plaintext.tobytes()
    e[buf_out] = b"\x00" * 8  # Need to do this so this buffer is mapped into unicorn
    
    e["r0"] = buf_in
    e["r1"] = key_addr    
    e["r2"] = buf_out
    e.start(e.functions["des_encrypt"] | 1, 0)

    # Hamming weight + noise to pretend we're on a real target
    trace = np.array([event["register"] for event in e.trace]) + np.random.normal(
        0, 1.0, (len(e.trace))
    )
    return trace, np.frombuffer(e[buf_out:buf_out+8], dtype='uint8')

In [4]:
plaintext = np.array([0x48, 0x45, 0x4c, 0x4f, 0x42, 0x55, 0x44, 0x59], dtype='uint8')
key = np.array([0x11, 0x22, 0x33, 0x44, 0x55, 0x66, 0x77, 0x88], dtype='uint8')

In [5]:
trace, ciphertext = des_encrypt(key, plaintext)

In [6]:
" ".join(f"{i:02x}" for i in ciphertext)

'6b e3 35 0e b9 10 08 7f'

In [7]:
traces = []
for i in range(5):
    plaintext = np.random.randint(0, 256, (8,), np.uint8)
    trace, ciphertext = des_encrypt(key, plaintext)
    traces.append(trace)

In [9]:
CurvePlot(np.array(traces))

E:\Kerja\Projek\Cryptography\Git\kresca\Example\Simulator\DES\../../..\kresca\plot\curve_plot.py:20: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("gui qt5")


<CurvePlot (PyQt5) at 0x23153c46070>

## Make Traces

In [13]:
outfile = estraces.ETSWriter('../../../Traces/DES_1k.ets', overwrite=True)

In [14]:
total = 1000

for i in tqdm(range(total)):
    plaintext = np.random.randint(0, 256, (8,), np.uint8)
    trace, ciphertext = des_encrypt(key, plaintext)
    outfile.write_samples(trace, index=i)
    outfile.write_meta('plaintext', plaintext, index=i)
    outfile.write_meta('key', key, index=i)
    outfile.write_meta('ciphertext', ciphertext, index=i)
    
outfile.close()

C:\Users\Tes\miniconda3\envs\SCA\lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/1000 [00:00<?, ?it/s]

C:\Users\Tes\miniconda3\envs\SCA\lib\site-packages\estraces\formats\ets_writer.py:217: DeprecationWarning: This method is deprecated and will be removed in a future version. Use write_samples instead.
  warnings.warn('This method is deprecated and will be removed in a future version. Use write_samples instead.', DeprecationWarning)
